<a href="https://colab.research.google.com/github/dannyticknor/MM/blob/main/NEW_rev4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

#Import the data from Q1 and Q2
kw_df_2 = pd.read_csv('University of Phoenix Managed Campaigns_Q2 Scope Report_keywords_010422_300622.csv')
kw_df_1 = pd.read_csv('University of Phoenix Managed Campaigns_Q1 Scope Report_keywords_010122_310322.csv')

#We weill only be interested in the Keywords that have 0 RFI in Q1 so filtering here will make everything 
#run faster
kw_df_1 = kw_df_1[kw_df_1['RFI'] == 0]
#Same for Q2 we are only interested in those keywords that have more than 0 RFI
kw_df_2 = kw_df_2[kw_df_2['RFI'] > 0]

In [ ]:
#Finding legacies and targets
#I want to exclude them from the other df and do this calculation separately. 

legacies = pd.DataFrame()
targets = pd.DataFrame()

for row in range(0, len(kw_df_2)):
    if (kw_df_2['Labels'].iat[row].__contains__('legacy') and kw_df_2['Labels'].iat[row].__contains__('0601')):
        legacies = legacies.append([(kw_df_2.iloc[row, :])])
    elif (kw_df_2['Labels'].iat[row].__contains__('0601') and not kw_df_2['Labels'].iat[row].__contains__('legacy')):
        targets = targets.append([(kw_df_2.iloc[row, :])])
        
restructure_df = pd.DataFrame()

for row_legacies in range(0, len(legacies)):
    for row_target in range(0, len(targets)):
        if(
            (legacies['Keyword ID'].iat[row_legacies] == targets['Keyword ID'].iat[row_target])
          & (legacies['Match type'].iat[row_legacies] == targets['Match type'].iat[row_target]) 
          & (legacies['Campaign'].iat[row_legacies] == targets['Campaign'].iat[row_target])  
        ):
            row_df = {'Keyword ID' : [legacies['Keyword ID'].iat[row_legacies]],
                       'Match type' : [legacies['Match type'].iat[row_legacies]],
                       'Campaign' : [legacies['Campaign'].iat[row_legacies]],
                       'RFI' : [legacies['RFI'].iat[row_legacies] + targets['RFI'].iat[row_legacies]] }
            row_df = pd.DataFrame(row_df)
            restructure_df = restructure_df.append(row_df)   

#I only want to select those that have more than 0 RFI in Q2            
restructure_df = restructure_df[restructure_df['RFI'] > 0]  

#in case of duplicates 
restructure_df.drop_duplicates(inplace = True)

In [ ]:
#Counting new RFIs for the restructure DF

restructure_new_RFI_df = pd.DataFrame()

for row_Q1 in range(0, len(kw_df_1)):
    for row_Q2 in range(0, len(restructure_df)):
        if (
              (kw_df_1['RFI'].iat[row_Q1] == 0)
            & (kw_df_1['Keyword ID'].iat[row_Q1] == restructure_df['Keyword ID'].iat[row_Q2])
            & (kw_df_1['Campaign'].iat[row_Q1] == restructure_df['Campaign'].iat[row_Q2])
            & (kw_df_1['Match type'].iat[row_Q1] == restructure_df['Match type'].iat[row_Q2])
        ):
            restructure_new_RFI_df = restructure_new_RFI_df.append([restructure_df.iloc[row_Q2, :]])
            
#in case of duplicates             
restructure_new_RFI_df.drop_duplicates(inplace = True)  

restructure_new_RFI = restructure_new_RFI_df['RFI'].sum()

In [ ]:
new_df = pd.DataFrame()

for row_Q2 in range(0, len(kw_df_2)):
    for row_Q1 in range(0, len(kw_df_1)):
        if(  
          (kw_df_2['Keyword ID'].iat[row_Q2] == kw_df_1['Keyword ID'].iat[row_Q1])
        & (kw_df_2['Campaign'].iat[row_Q2] == kw_df_1['Campaign'].iat[row_Q1])
        & (kw_df_2['Match type'].iat[row_Q2] == kw_df_1['Match type'].iat[row_Q1])
        & (kw_df_2['Ad group'].iat[row_Q2] == kw_df_1['Ad group'].iat[row_Q1])    
        ):
            new_df = new_df.append([kw_df_2.iloc[row_Q2, :]])  

In [ ]:
new_df = pd.DataFrame()

for row_Q2 in range(0, len(kw_df_2)):
    for row_Q1 in range(0, len(kw_df_1)):
        if(  
          (kw_df_2['Keyword ID'].iat[row_Q2] == kw_df_1['Keyword ID'].iat[row_Q1])
        & (kw_df_2['Campaign'].iat[row_Q2] == kw_df_1['Campaign'].iat[row_Q1])
        & (kw_df_2['Match type'].iat[row_Q2] == kw_df_1['Match type'].iat[row_Q1])
        & (kw_df_2['Ad group'].iat[row_Q2] == kw_df_1['Ad group'].iat[row_Q1])
            
        ):
            new_df = new_df.append([kw_df_2.iloc[row_Q2, :]])
            
#I haven't excluded yet the results from the restructure so dropping all the duplicates
new_df.drop_duplicates(inplace = True)  

In [ ]:
pd.set_option('display.max_rows', None)

new_df[['RFI','Keyword', 'Keyword ID','Campaign', 'Ad group','Match type', 'Labels']].sort_values('RFI', ascending = False)

,RFI,Keyword,Keyword ID,Campaign,Ad group,Match type,Labels
57,47.504253,online programs for college,488402482992,!College Online RS - 8 States,College - Online Programs,Broad,--
610,17.001544,taking college classes online,4309184013,!Classes Online RS,classes-online-college-take,Exact,"[""MM-ako-012622""]"
197,15.000000,courses online,10018266,!Courses Online RS,Online Courses,Broad,"[""Broad Exp""]"
255,15.000000,online college,10579091,!College Online RS - 8 States,College - Online,Broad,--
228,13.844826,courses for school online,923226422313,!School Online RS,School - Online Courses,Broad,"[""Broad Exp""]"
147,13.843411,associates courses,297109328399,!Associates RS,associates-courses,Exact,"[""MM-ako-060122""]"
297,12.500000,online college,10579091,!College Online RS,College - Online,Broad,--
389,12.498336,college programs online,132677358,!College Online RS - 8 States,College - Online Programs,Broad,--
293,12.000000,online degree college,14540287,!College Online RS - 8 States,College - Online Degree,Broad,--
224,11.500000,find online college,158916885,!College Online RS - 8 States,College - Online Find,Broad,--


In [ ]:
RFI_total = restructure_new_RFI + new_df['RFI'].sum()

In [ ]:
RFI_total*1444.77

1214791.7714586

In [ ]:
RFI_total

840.8201799999999

In [ ]:
#Making sure numbers match. The new_df matches keywords to their ad group so it should naturally exclude the ones
#that were part of the restructure, but this is just an extra check. 
#In this case the numbers are the same
all_df = pd.DataFrame()

all_df = all_df.append(restructure_new_RFI_df[['RFI', 'Keyword ID','Campaign','Match type']])
all_df = all_df.append(new_df[['RFI', 'Keyword ID','Campaign','Match type']])

all_df.drop_duplicates(inplace = True)

print(all_df['RFI'].sum()*1444.77)

1214791.7714586
